# Purpose

Figure out pulling location IDs for only DCFC US (non-Tesla to start with) in the US. Need location IDs to efficiently scrape Plugshare with some of our other code.

# Imports

In [1]:

%load_ext autoreload
%autoreload 2

import numpy as np
from rich import print
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from typing import List, Union, Set

# from torch.utils.data import DataLoader

from evlens.data.plugshare import MainMapScraper
from evlens.data.google_cloud import BigQuery

# Electrify America in Springfield, VA mall parking lot
TEST_LOCATION = 252784

from dotenv import load_dotenv
load_dotenv(override=True)

from evlens.logs import setup_logger
logger = setup_logger("Notebook-0.3")
logger.info("TEST!")

2024-07-04_T10_30_21EDT: INFO (Notebook-0.3:L26) - TEST!


# Set up our constants

In [2]:
# URL = "https://developer.plugshare.com/embed"
location_id = 252784
URL = f"https://www.plugshare.com/location/{location_id}"

In [3]:
# Load up some example scrapes

from joblib import load
test_results = load("../../test_results.pkl")
df_stations, df_checkins = test_results[0]
df_checkins.drop_duplicates(inplace=True)
df_checkins.info()
df_checkins.head()

<class 'pandas.core.frame.DataFrame'>
Index: 49 entries, 0 to 49
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    49 non-null     datetime64[ns]
 1   car                     49 non-null     object        
 2   connector_type          28 non-null     object        
 3   charge_power_kilowatts  29 non-null     object        
 4   problem                 4 non-null      object        
 5   comment                 27 non-null     object        
 6   location_id             49 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 3.1+ KB


,date,car,connector_type,charge_power_kilowatts,problem,comment,location_id
0,2024-06-30,Genesis GV60 2023,CCS/SAE,245 Kilowatts,NaN,NaN,252784
1,2024-06-29,Kia EV6 2024,CCS/SAE,242 Kilowatts,NaN,NaN,252784
2,2024-06-22,Kia EV6 2022,CCS/SAE,230 Kilowatts,NaN,NaN,252784
3,2024-06-21,Chevrolet Bolt EV 2017,J-1772,NaN,NaN,NaN,252784
4,2024-06-16,Hyundai Ioniq 5 2024,CCS/SAE,242 Kilowatts,NaN,NaN,252784


# Try uploading a screenshot to GCP Cloud Storage

This isn't datatabase stuff but useful for scraping and *should* be easy to do...

In [7]:
from evlens.data.google_cloud import upload_file

upload_file(
    bucket_name='plugshare_scraping',
    source_filepath='../../data/external/plugshare/06-30-2024/errors/07-01-2024_T04_37_46_checkins.png', 
    destination_blob_name='errors/07-01-2024_T04_37_46_checkins.png'
)

2024-07-01_T17_53_05CDT: INFO (evlens.data.google_cloud:L41) - File ../../data/external/plugshare/06-30-2024/errors/07-01-2024_T04_37_46_checkins.png uploaded to errors/07-01-2024_T04_37_46_checkins.png.


In [5]:
# Push our hex data for location ID scraping to the bucket
from evlens.data.google_cloud import upload_file

source_path = "/Users/davemcrench/Projects/evlens/data/processed/h3_hexagons_usa.pkl"

upload_file(
    bucket_name='plugshare_scraping',
    source_filepath=source_path,
    destination_blob_name='location_id_scraping/h3_hexagons_usa.pkl'
)

ConnectionError: ('Connection aborted.', TimeoutError('The write operation timed out'))

# Build Out BigQuery Functionality

In [9]:
from evlens.data.google_cloud import BigQuery

bq = BigQuery()
bq.create_dataset('plugshare')

2024-07-04_T10_51_16EDT: WARNING (google.auth._default:L683) - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/evlens/datasets?prettyPrint=false: Already Exists: Dataset evlens:plugshare

In [10]:
bq = BigQuery()
bq.list_datasets()

2024-07-04_T10_51_22EDT: WARNING (google.auth._default:L683) - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
2024-07-04_T10_51_23EDT: INFO (evlens.data.google_cloud:L133) - Datasets in project evlens: ['plugshare']


In [8]:
bq.setup_table('plugshare', 'locationID', '../../cloud/bq_plugshare_locationIDTable_config.json')

2024-07-03_T21_53_25EDT: INFO (evlens.data.google_cloud:L153) - Created table evlens.plugshare.locationID.


In [9]:
bq.setup_table('plugshare', 'checkins', '../../cloud/bq_plugshare_checkinsTable_config.json')

2024-07-03_T21_54_10EDT: INFO (evlens.data.google_cloud:L153) - Created table evlens.plugshare.checkins.


In [10]:
bq.setup_table('plugshare', 'stations', '../../cloud/bq_plugshare_stationsTable_config.json')

2024-07-03_T21_54_31EDT: INFO (evlens.data.google_cloud:L153) - Created table evlens.plugshare.stations.


In [14]:
# Add some key assignments
# bq = BigQuery()
bq.set_table_keys(
    'plugshare',
    'locationID',
    "id",
    foreign_keys=None
)

QueryJob<project=evlens, location=US, id=e0fd7076-2a50-41f1-98a6-a0940964a83f>

In [15]:
# Add some key assignments
# bq = BigQuery()
bq.set_table_keys(
    'plugshare',
    'stations',
    "id",
    foreign_keys=[
        {
            'key': 'location_id',
            'foreign_table': bq._make_table_id('plugshare', 'locationID'),
            'foreign_column': 'id'
        }
    ]
)

QueryJob<project=evlens, location=US, id=cdb0902f-0f12-4308-9631-1715c699f3c7>

In [16]:
# Add some key assignments
# bq = BigQuery()
bq.set_table_keys(
    'plugshare',
    'checkins',
    "id",
    foreign_keys=[
        {
            'key': 'station_id',
            'foreign_table': bq._make_table_id('plugshare', 'stations'),
            'foreign_column': 'id'
        }
    ]
)

QueryJob<project=evlens, location=US, id=c4d455c5-8ff3-46ee-8146-caea1d73034d>

## Inserting Data into Checkins Table

In [45]:
import re

def _get_power_number(text: str) -> int:
    '''
    Extracts the value from a power string. E.g. "110 Kilowatts" returns the integer 110.

    Parameters
    ----------
    text : str
        The text to extract the leading number from

    Returns
    -------
    int
        The value in the string
    '''
    if text is None or pd.isna(text):
        return np.nan
    elif isinstance(text, (int, float)):
        return text
    
    match = re.search(r"\d+", text)
    if match:
        return int(match.group(0))
    return np.nan

In [50]:
# Write data to BQ checkins table
df_checkins = test_results[0][1]
df_checkins['id'] = [bq._make_uuid() for _ in range(len(df_checkins))]
df_checkins.rename(columns={'location_id': 'station_id'}, inplace=True)
df_checkins['charge_power_kilowatts'] = df_checkins['charge_power_kilowatts'].apply(_get_power_number)
df_checkins['station_id'] = df_checkins['station_id'].astype(str)
df_checkins.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49 entries, 0 to 49
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    49 non-null     datetime64[ns]
 1   car                     49 non-null     object        
 2   connector_type          28 non-null     object        
 3   charge_power_kilowatts  29 non-null     float64       
 4   problem                 4 non-null      object        
 5   comment                 27 non-null     object        
 6   station_id              49 non-null     object        
 7   id                      49 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 4.5+ KB


In [47]:
query = f"""
SELECT *
FROM `{bq._make_table_id('plugshare', 'checkins')}`
"""
bq.query_to_dataframe(query)

,id,date,car,problem,connector_type,charge_power_kilowatts,comment,station_id


In [56]:
result = bq.insert_data(
    df_checkins,
    'plugshare',
    'checkins'
)

query = f"""
SELECT *
FROM `{bq._make_table_id('plugshare', 'checkins')}`
"""
df_test = bq.query_to_dataframe(query)
df_test.info()
df_test.sample(5)

2024-07-04_T11_38_10EDT: INFO (evlens.data.google_cloud:L262) - Loaded 245 rows and 8 columns to evlens.plugshare.checkins


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      245 non-null    object
 1   date                    245 non-null    dbdate
 2   car                     245 non-null    object
 3   problem                 20 non-null     object
 4   connector_type          140 non-null    object
 5   charge_power_kilowatts  145 non-null    Int64 
 6   comment                 135 non-null    object
 7   station_id              245 non-null    object
dtypes: Int64(1), dbdate(1), object(6)
memory usage: 15.7+ KB


,id,date,car,problem,connector_type,charge_power_kilowatts,comment,station_id
141,c2539c56-09ad-4eb9-85ad-ba3802e65145,2024-03-15,Ford Mustang Mach-E 2022,NaN,CCS/SAE,156,"Tried #2, charged 4 minutes, stopped. Tried ag...",252784
94,dbe404d1-b726-41ea-8c8a-947bb4cbfa80,2024-04-20,Hyundai Ioniq 5 2022,BROKEN HARDWARE,NaN,<NA>,All stations down,252784
39,6457d145-f3cf-40cb-b4a0-d04e80c7f3df,2024-03-25,Ford Mustang Mach-E 2022,NaN,CCS/SAE,90,Only able to pull 90 kw max and that leveled o...,252784
133,7be05420-79e5-4e9c-81ad-908cd9bfd136,2024-04-08,Volkswagen ID.4 2023,NaN,CCS/SAE,<NA>,NaN,252784
86,72b27d1b-6499-4cef-860e-8264e3b308cf,2024-04-06,Volkswagen ID.4 2023,NaN,CCS/SAE,194,NaN,252784


In [57]:
bq.clear_table('plugshare', 'checkins')

query = f"""
SELECT *
FROM `{bq._make_table_id('plugshare', 'checkins')}`
"""
df_test = bq.query_to_dataframe(query)
df_test.info()

2024-07-04_T11_38_34EDT: INFO (evlens.data.google_cloud:L278) - Table evlens.plugshare.checkins cleared


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      0 non-null      object
 1   date                    0 non-null      object
 2   car                     0 non-null      object
 3   problem                 0 non-null      object
 4   connector_type          0 non-null      object
 5   charge_power_kilowatts  0 non-null      object
 6   comment                 0 non-null      object
 7   station_id              0 non-null      object
dtypes: object(8)
memory usage: 132.0+ bytes


## Inserting Data into Stations Table

In [58]:
query = f"""
SELECT *
FROM `{bq._make_table_id('plugshare', 'stations')}`
"""
bq.query_to_dataframe(query)

,id,name,address,plugscore,wattage,service_hours,checkin_count,location_id


In [68]:
# Write data to BQ stations table
df_stations = test_results[0][0].drop(columns=['id']).drop_duplicates()
df_stations['id'] = [BigQuery.make_uuid() for _ in range(len(df_stations))]
df_stations['location_id'] = str(TEST_LOCATION)
df_stations['plugscore'] = df_stations['plugscore'].astype(float)
# df_stations.rename(columns={'location_id': 'station_id'}, inplace=True)
# df_stations['station_id'] = df_checkins['station_id'].astype(str)
df_stations.info()
df_stations.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           1 non-null      object 
 1   address        1 non-null      object 
 2   plugscore      1 non-null      float64
 3   wattage        1 non-null      object 
 4   service_hours  1 non-null      object 
 5   checkin_count  1 non-null      int64  
 6   id             1 non-null      object 
 7   location_id    1 non-null      object 
dtypes: float64(1), int64(1), object(6)
memory usage: 72.0+ bytes


,name,address,plugscore,wattage,service_hours,checkin_count,id,location_id
0,Springfield Town Center - Target - East Lot (1),"6600 Springfield Mall, Springfield, Virginia, ...",10.0,7 - 350 kW,Open 24/7,450,e51dd15f-d1cb-423c-9ef9-571a9c526188,252784


In [69]:
table_name = 'stations'

result = bq.insert_data(
    df_stations,
    'plugshare',
    table_name
)

query = f"""
SELECT *
FROM `{bq._make_table_id('plugshare', table_name)}`
"""
df_test = bq.query_to_dataframe(query)
df_test.info()
df_test.head(5)

2024-07-04_T11_55_41EDT: INFO (evlens.data.google_cloud:L263) - Loaded 1 rows and 8 columns to evlens.plugshare.stations


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1 non-null      object 
 1   name           1 non-null      object 
 2   address        1 non-null      object 
 3   plugscore      1 non-null      float64
 4   wattage        1 non-null      object 
 5   service_hours  1 non-null      object 
 6   checkin_count  1 non-null      Int64  
 7   location_id    1 non-null      object 
dtypes: Int64(1), float64(1), object(6)
memory usage: 197.0+ bytes


ValueError: Cannot take a larger sample than population when 'replace=False'

In [70]:
df_test.head(5)

,id,name,address,plugscore,wattage,service_hours,checkin_count,location_id
0,e51dd15f-d1cb-423c-9ef9-571a9c526188,Springfield Town Center - Target - East Lot (1),"6600 Springfield Mall, Springfield, Virginia, ...",10.0,7 - 350 kW,Open 24/7,450,252784


In [63]:
test_results[0][0]

,name,address,plugscore,wattage,service_hours,checkin_count,id
0,Springfield Town Center - Target - East Lot (1),"6600 Springfield Mall, Springfield, Virginia, ...",10,7 - 350 kW,Open 24/7,450,a42bd13a-4480-454d-acdf-f5815fb1cae2
1,Springfield Town Center - Target - East Lot (1),"6600 Springfield Mall, Springfield, Virginia, ...",10,7 - 350 kW,Open 24/7,450,dd136ee1-84b7-4903-a640-d14e8c6e7abc


## Loading in Hexagon Data

In [75]:
import geopandas as gpd

df_hex = pd.read_pickle('../../data/processed/h3_hexagons_usa.pkl')
df_hex.info()
df_hex.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1812316 entries, 872631cacffffff to 872b99662ffffff
Data columns (total 5 columns):
 #   Column     Dtype   
---  ------     -----   
 0   geometry   geometry
 1   cell_area  float64 
 2   lat_long   object  
 3   latitude   float64 
 4   longitude  float64 
dtypes: float64(3), geometry(1), object(1)
memory usage: 83.0+ MB


,geometry,cell_area,lat_long,latitude,longitude
h3_polyfill,,,,,
872631cacffffff,"POLYGON ((-98.89411 45.47622, -98.90505 45.466...",2.011903,"(45.4644308268348, -98.88678418817953)",45.464431,-98.886784
872aad931ffffff,"POLYGON ((-74.65316 39.46713, -74.6674 39.4605...",1.956755,"(39.45501532030005, -74.65252127744267)",39.455015,-74.652521
872a91d64ffffff,"POLYGON ((-82.96187 39.80834, -82.97493 39.800...",1.960595,"(39.79630739720016, -82.95910787730979)",39.796307,-82.959108
870c2e831ffffff,"POLYGON ((-153.77099 65.74139, -153.79115 65.7...",1.961247,"(65.7299171835115, -153.760950462319)",65.729917,-153.760950
8729b0666ffffff,"POLYGON ((-113.61065 34.98822, -113.60209 34.9...",2.262247,"(34.99986610719769, -113.61867862603036)",34.999866,-113.618679


In [74]:
df_hex.shape

(1812316, 5)

In [79]:
df_hex.reset_index(inplace=True)
df_hex.info()
df_hex.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1812316 entries, 0 to 1812315
Data columns (total 6 columns):
 #   Column       Dtype   
---  ------       -----   
 0   h3_polyfill  object  
 1   geometry     geometry
 2   cell_area    float64 
 3   lat_long     object  
 4   latitude     float64 
 5   longitude    float64 
dtypes: float64(3), geometry(1), object(2)
memory usage: 83.0+ MB


,h3_polyfill,geometry,cell_area,lat_long,latitude,longitude
0,872631cacffffff,"POLYGON ((-98.89411 45.47622, -98.90505 45.466...",2.011903,"(45.4644308268348, -98.88678418817953)",45.464431,-98.886784
1,872aad931ffffff,"POLYGON ((-74.65316 39.46713, -74.6674 39.4605...",1.956755,"(39.45501532030005, -74.65252127744267)",39.455015,-74.652521
2,872a91d64ffffff,"POLYGON ((-82.96187 39.80834, -82.97493 39.800...",1.960595,"(39.79630739720016, -82.95910787730979)",39.796307,-82.959108
3,870c2e831ffffff,"POLYGON ((-153.77099 65.74139, -153.79115 65.7...",1.961247,"(65.7299171835115, -153.760950462319)",65.729917,-153.760950
4,8729b0666ffffff,"POLYGON ((-113.61065 34.98822, -113.60209 34.9...",2.262247,"(34.99986610719769, -113.61867862603036)",34.999866,-113.618679


In [80]:
# Setup the table
table_name = 'searchTiles'

bq.setup_table('plugshare', table_name, '../../cloud/bq_plugshare_searchTilesTable_config.json')

2024-07-04_T12_05_59EDT: INFO (evlens.data.google_cloud:L173) - Created table evlens.plugshare.searchTiles.


In [82]:
query = f"""
SELECT *
FROM `{bq._make_table_id('plugshare', table_name)}`
"""
df_test = bq.query_to_dataframe(query)
df_test.info()
df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              0 non-null      object
 1   h3_polyfill     0 non-null      object
 2   cell_radius_mi  0 non-null      object
 3   latitude        0 non-null      object
 4   longitude       0 non-null      object
dtypes: object(5)
memory usage: 132.0+ bytes


,id,h3_polyfill,cell_radius_mi,latitude,longitude


In [88]:
df_hex['cell_radius_mi'] = np.sqrt(2 * df_hex['cell_area'] / (3 * np.sqrt(3)))

0    0.879990
1    0.867845
2    0.868696
3    0.868841
4    0.933134
Name: cell_area, dtype: float64

In [89]:
# Massage a bit more to match schema
df_hex['id'] = [BigQuery.make_uuid() for _ in range(len(df_hex))]
df_hex['cell_radius_mi'] = np.sqrt(2 * df_hex['cell_area'] / (3 * np.sqrt(3)))
df_hex.drop(columns=['geometry', 'cell_area', 'lat_long'], inplace=True)
df_hex.info()
df_hex.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1812316 entries, 0 to 1812315
Data columns (total 5 columns):
 #   Column          Dtype  
---  ------          -----  
 0   h3_polyfill     object 
 1   latitude        float64
 2   longitude       float64
 3   id              object 
 4   cell_radius_mi  float64
dtypes: float64(3), object(2)
memory usage: 69.1+ MB


,h3_polyfill,latitude,longitude,id,cell_radius_mi
0,872631cacffffff,45.464431,-98.886784,811d4a85-037a-4616-b944-2559c96ae459,0.879990
1,872aad931ffffff,39.455015,-74.652521,87da4db7-adb7-4959-9a32-a687c737416a,0.867845
2,872a91d64ffffff,39.796307,-82.959108,6394025e-d327-4cd7-bcd4-1f87f295ef09,0.868696
3,870c2e831ffffff,65.729917,-153.760950,4c2d3a79-bc76-4227-83d3-27cb250ec92a,0.868841
4,8729b0666ffffff,34.999866,-113.618679,59e77095-7f9a-47af-918c-7cf66abca1c4,0.933134


In [91]:
df_hex['cell_radius_mi'].describe()

count    1.812316e+06
mean     8.823830e-01
std      3.479788e-02
min      7.763017e-01
25%      8.592466e-01
50%      8.831516e-01
75%      9.070450e-01
max      9.532796e-01
Name: cell_radius_mi, dtype: float64

In [92]:
bq.clear_table('plugshare', table_name)

result = bq.insert_data(
    df_hex,
    'plugshare',
    table_name
)

2024-07-04_T12_15_31EDT: INFO (evlens.data.google_cloud:L279) - Table evlens.plugshare.searchTiles cleared
2024-07-04_T12_19_18EDT: INFO (evlens.data.google_cloud:L263) - Loaded 1812316 rows and 5 columns to evlens.plugshare.searchTiles


In [93]:
query = f"""
SELECT *
FROM `{bq._make_table_id('plugshare', table_name)}`
LIMIT 5
"""
df_test = bq.query_to_dataframe(query)
df_test.info()
df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              5 non-null      object 
 1   h3_polyfill     5 non-null      object 
 2   cell_radius_mi  5 non-null      float64
 3   latitude        5 non-null      float64
 4   longitude       5 non-null      float64
dtypes: float64(3), object(2)
memory usage: 332.0+ bytes


,id,h3_polyfill,cell_radius_mi,latitude,longitude
0,811d4a85-037a-4616-b944-2559c96ae459,872631cacffffff,0.879990,45.464431,-98.886784
1,87da4db7-adb7-4959-9a32-a687c737416a,872aad931ffffff,0.867845,39.455015,-74.652521
2,6394025e-d327-4cd7-bcd4-1f87f295ef09,872a91d64ffffff,0.868696,39.796307,-82.959108
3,4c2d3a79-bc76-4227-83d3-27cb250ec92a,870c2e831ffffff,0.868841,65.729917,-153.760950
4,59e77095-7f9a-47af-918c-7cf66abca1c4,8729b0666ffffff,0.933134,34.999866,-113.618679


In [94]:
query = f"""
SELECT COUNT(id)
FROM `{bq._make_table_id('plugshare', table_name)}`
"""
df_test = bq.query_to_dataframe(query)
df_test

,f0_
0,1812316


In [96]:
df_test.iloc[0,0] == len(df_hex)

np.True_

## Loading Data into locationID table

In [71]:
query = f"""
SELECT *
FROM `{bq._make_table_id('plugshare', 'locationID')}`
"""
bq.query_to_dataframe(query)

,id,parsed_datetime,plug_types_searched,location_id,search_cell_latitude,search_cell_longitude
